# Zillow Challenge

##  Data input

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numpy import seterr,isneginf,array
from datetime import datetime
from pandas import compat
from operator import itemgetter
from sklearn import tree
from sklearn import metrics
from sklearn.preprocessing import scale
from sklearn.neighbors import KDTree
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import export_graphviz
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from IPython.display import Image

#import seaborn as sns

%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', 200)
pd.options.display.float_format = '{:20,.2f}'.format
compat.PY3 = False

def Decision_Tree_Image(decision_tree, feature_names, name="temp"):
    
    # Etrainport our decision tree to graphviz format
    dot_file = tree.export_graphviz(decision_tree, out_file='images/' + name + '.dot', feature_names=feature_names)
    
    # Call graphviz to make an image file from our decision tree
    os.system("dot -T png images/" + name + ".dot -o images/" + name + ".png")
    
    # Return the .png image so we can see it
    return Image(filename='images/' + name + '.png')

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - An object to be used as a cross-validation generator.
          - An iterable yielding train/test splits.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : integer, optional
        Number of jobs to run in parallel (default 1).
    """
    plt.figure()
    plt.title(title)
    #if ylim is not None:
    #    plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, scoring='neg_mean_absolute_error', train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [2]:
train_f = "train_2016_v2.csv"
train = pd.read_csv(train_f, index_col = 'parcelid', parse_dates=['transactiondate'])
train['transactiondate'] = pd.to_datetime(train['transactiondate']).astype(int)

properties_f = "properties_2016.csv"
properties = pd.read_csv(properties_f, index_col = 'parcelid')

train_bak = train.join(properties) 

/home/ec2-user/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#### TEST MODE  #####
def getdata(pcnt): 
    return train_bak.sample(frac=pcnt)

In [18]:
train = getdata(1)

##  Feature creation

In [19]:
#################
#  taxrate
#################
def add_taxrate(dfs):
    dfs[0]['taxrate'] = np.divide(dfs[0]['taxamount'],dfs[0]['taxvaluedollarcnt'])

In [20]:
#################
# add_transactioncnt
#################

def add_transactioncnt(dfs, features, monthlag):
    name = 'transactioncnt'+str(monthlag)
    
    n1 = 200  # calculatedfinishedsquarefeet
    n2 = 50000 # taxvaluedollarcnt
    
    dfs[0].drop([name], errors='ignore', axis=1, inplace = True)

    (dfs[0])['transactionmonth']    = pd.to_datetime((dfs[0])['transactiondate']).dt.month
    (dfs[0])['transactionmonthlag'] = np.subtract((dfs[0])['transactionmonth'],monthlag)    
    
    (dfs[0])['calculatedfinishedsquarefeet_t'] = np.ceil(np.divide((dfs[0])['calculatedfinishedsquarefeet'], n1))
    (dfs[0])['taxvaluedollarcnt_t'] = np.ceil(np.divide((dfs[0])['taxvaluedollarcnt'], n2))
    
    sumdf = pd.DataFrame({name : dfs[0].groupby(['transactionmonth']+features).size()}).reset_index()
    sumdf.dropna(axis=1, how='any', inplace=True)
    sumdf.rename(columns={'transactionmonth': 'transactionmonthlag'}, inplace=True)
    sumdf.set_index(['transactionmonthlag']+features, inplace=True)
    
    dfs[:] = [(dfs[0]).join(sumdf, on=['transactionmonthlag']+features)]
    (dfs[0]).drop(['transactionmonth',
                   'transactionmonthlag',
                   'calculatedfinishedsquarefeet_t',
                   'taxvaluedollarcnt_t'
                  ], axis=1, inplace = True)

In [21]:
#Binary
features_binary = {'hashottuborspa',
                'pooltypeid10',
                'pooltypeid2',
                'pooltypeid7',
                'fireplaceflag',
                'taxdelinquencyflag'}

# Types
features_types = {'airconditioningtypeid',
                  'architecturalstyletypeid',
                  'buildingqualitytypeid',
                  'buildingclasstypeid',
                  'decktypeid',
                  'heatingorsystemtypeid',
                  'propertycountylandusecode',
                  'propertylandusetypeid',
                  'propertyzoningdesc',
                  'typeconstructiontypeid',
                  'storytypeid'}

In [22]:
def createfeatures(dfs, features):
    functions = {
                'taxrate': add_taxrate,               
                } 
    for f in features:
        functions[f](dfs)
        
    features = ['bedroomcnt',
            'bathroomcnt',
            'taxvaluedollarcnt_t',
            'regionidzip'
           ]
    
    add_transactioncnt(dfs, features, 3)
    add_transactioncnt(dfs, features, 4)
    add_transactioncnt(dfs, features, 5)    

In [23]:
dfs = [train]
createfeatures(dfs,[
               'taxrate'
              ])
train = dfs[0]

for f in features_binary:
    train[f] = pd.Categorical(train[f]).codes

for f in features_types:
    train[f] = pd.Categorical(train[f]).codes

In [24]:
train = train.fillna(-1)
train_full = train

In [25]:
train.columns

Index([u'logerror', u'transactiondate', u'airconditioningtypeid',
       u'architecturalstyletypeid', u'basementsqft', u'bathroomcnt',
       u'bedroomcnt', u'buildingclasstypeid', u'buildingqualitytypeid',
       u'calculatedbathnbr', u'decktypeid', u'finishedfloor1squarefeet',
       u'calculatedfinishedsquarefeet', u'finishedsquarefeet12',
       u'finishedsquarefeet13', u'finishedsquarefeet15',
       u'finishedsquarefeet50', u'finishedsquarefeet6', u'fips',
       u'fireplacecnt', u'fullbathcnt', u'garagecarcnt', u'garagetotalsqft',
       u'hashottuborspa', u'heatingorsystemtypeid', u'latitude', u'longitude',
       u'lotsizesquarefeet', u'poolcnt', u'poolsizesum', u'pooltypeid10',
       u'pooltypeid2', u'pooltypeid7', u'propertycountylandusecode',
       u'propertylandusetypeid', u'propertyzoningdesc',
       u'rawcensustractandblock', u'regionidcity', u'regionidcounty',
       u'regionidneighborhood', u'regionidzip', u'roomcnt', u'storytypeid',
       u'threequarterbathnbr', u

##  Feature selection

In [26]:
features_to_keep = set([
'logerror',
'transactiondate',
'bedroomcnt',
 'buildingqualitytypeid',
 'calculatedbathnbr',
 'calculatedfinishedsquarefeet',
 'censustractandblock',
 'finishedfloor1squarefeet',
 'finishedsquarefeet12',
 'finishedsquarefeet50',
 'fullbathcnt',
 'garagetotalsqft',
 'landtaxvaluedollarcnt',
 'latitude',
 'longitude',
 'lotsizesquarefeet',
 'poolcnt',
 'propertycountylandusecode',
 'propertylandusetypeid',
 'propertyzoningdesc',
 'rawcensustractandblock',
 'regionidcity',
 'regionidneighborhood',
 'regionidzip',
 'structuretaxvaluedollarcnt',
 'taxamount',
 'taxrate',
 'taxvaluedollarcnt',
 'threequarterbathnbr',
 'transactioncnt3',
 'transactioncnt4',
 'transactioncnt5',
 'yearbuilt'
        ])

In [27]:
train_filtered = train_full
for column in train_full.columns:
    if column not in features_to_keep:
        train_filtered = train_filtered.drop(column, axis=1)

## Sample

In [28]:
def getsample(pcnt): 
    return train_filtered.sample(frac=pcnt)

In [76]:
train_sample = getsample(1)

In [77]:
train_sample['transactionmonth'] = pd.to_datetime(train_sample['transactiondate']).dt.month
train_sample = train_sample[train_sample['transactionmonth'] > 6]
train_sample.drop(['transactionmonth', 'transactiondate'], axis=1, inplace=True)

In [78]:
Y = train_sample[['logerror']]
X = train_sample.drop('logerror',axis=1)

## Modeling

In [ ]:
import math 

max_depth = None
min_samples_split = int(round(len(X)/6, 0))
min_samples_leaf = int(round(len(X)/80, 0))

print("count: ", len(train_sample))
print("min_samples_split =", min_samples_split)
print("min_samples_leaf =", min_samples_leaf)

n_jobs = int(len(X.columns)/2)
model = RandomForestRegressor(criterion = "mae",
                             #min_samples_split = min_samples_split,
                              min_samples_leaf = min_samples_leaf,
                              n_jobs = n_jobs,
                              n_estimators = n_jobs, warm_start = False
                              )


In [ ]:
# Fit regression model
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=.8)
model.fit(X_train, Y_train.values.ravel())

In [ ]:
# Feature importance
sorted(zip(map(lambda x: round(x, 4), model.feature_importances_), X_train.columns), 
             reverse=True)

In [ ]:
# Zillow MAE (test)
Y_predict=model.predict(X_test)
print("MAE: ",metrics.mean_absolute_error(Y_test['logerror'], Y_predict))


## Model Execution

In [147]:
sample_submission_f = 'sample_submission.csv'
submission = pd.read_csv(sample_submission_f, index_col='ParcelId')

In [148]:
# Prediction transaction date
def add_transactiondate(df, yy,mm,dd):
    df['transactiondate'] = pd.datetime(yy,mm,dd)
    df['transactiondate'] = df['transactiondate'].astype(int)

In [149]:
def predict(df, yy,mm,dd):
    X_all = properties
    add_transactiondate(X_all, yy, mm, dd)
    
    # Add features
    dfs = [X_all]
    # Add custom features
    createfeatures(dfs,[
                   'taxrate',
                  ])
    X_all = dfs[0]

    for f in features_binary:
        X_all[f] = pd.Categorical(X_all[f]).codes

    for f in features_types:
        X_all[f] = pd.Categorical(X_all[f]).codes

    X_all.fillna(-1, inplace=True)
    
    for column in X_all.columns:
        if column not in features_to_keep:
            X_all.drop(column, axis=1, inplace=True)
    X_all.drop('transactiondate', axis=1, inplace=True)
    return model.predict(X_all)

In [150]:
results = pd.DataFrame(index=X_all.index)
results.index.names = ['ParcelId']

Y_oct = predict(properties, 2016, 10, 1)
Y_nov = predict(properties, 2016, 11, 1)
Y_dec = predict(properties, 2016, 12, 1)

results['201610'] = Y_oct
results['201611'] = Y_nov 
results['201612'] = Y_dec 
results['201710'] = Y_oct
results['201711'] = Y_nov 
results['201712'] = Y_dec 

In [151]:
results['201612'].describe()

count        2,985,217.00000
mean                 0.00844
std                  0.00727
min                 -0.01466
25%                  0.00395
50%                  0.00834
75%                  0.01291
max                  0.03044
Name: 201612, dtype: float64

In [152]:
submission = submission.drop(submission.columns[0:], axis=1)
submission = submission.join(results)

In [153]:
# Round as per rules
submission = submission.round(4)

In [154]:
submission

,201610,201611,201612,201710,201711,201712
ParcelId,,,,,,
10754147,0.01590,0.01590,0.01590,0.01590,0.01590,0.01590
10759547,0.01580,0.01580,0.01580,0.01580,0.01580,0.01580
10843547,0.00870,0.00870,0.00870,0.00870,0.00870,0.00870
10859147,0.01410,0.01410,0.01410,0.01410,0.01410,0.01410
10879947,0.00660,0.00660,0.00660,0.00660,0.00660,0.00660
10898347,0.00760,0.00760,0.00760,0.00760,0.00760,0.00760
10933547,0.00940,0.00940,0.00940,0.00940,0.00940,0.00940
10940747,0.01040,0.01040,0.01040,0.01040,0.01040,0.01040
10954547,0.01630,0.01630,0.01630,0.01630,0.01630,0.01630


## Sanity Checks

In [155]:
# Average log error
pd.options.display.float_format = '{:20,.5f}'.format
submission.describe()

,201610,201611,201612,201710,201711,201712
count,"2,985,217.00000","2,985,217.00000","2,985,217.00000","2,985,217.00000","2,985,217.00000","2,985,217.00000"
mean,0.00844,0.00843,0.00844,0.00844,0.00843,0.00844
std,0.00727,0.00726,0.00727,0.00727,0.00726,0.00727
min,-0.01470,-0.01470,-0.01470,-0.01470,-0.01470,-0.01470
25%,0.00400,0.00390,0.00400,0.00400,0.00390,0.00400
50%,0.00830,0.00830,0.00830,0.00830,0.00830,0.00830
75%,0.01290,0.01290,0.01290,0.01290,0.01290,0.01290
max,0.03040,0.03040,0.03040,0.03040,0.03040,0.03040


In [156]:
# Check for NaN
submission.isnull().sum()

201610    0
201611    0
201612    0
201710    0
201711    0
201712    0
dtype: int64

In [157]:
# Check if any duplicates
submission[submission.index.duplicated(keep=False)]

,201610,201611,201612,201710,201711,201712
ParcelId,,,,,,


In [158]:
# Check additional values in submission file
submission[~submission.index.isin(properties.index)]

,201610,201611,201612,201710,201711,201712
ParcelId,,,,,,


In [159]:
# Check additional values in properties file
properties[~properties.index.isin(submission.index)]

,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,transactiondate,taxrate,transactionmonth,transactionmonthlag,calculatedfinishedsquarefeet_t,taxvaluedollarcnt_t
parcelid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


## Dump File

In [160]:
# Write file
submission_f = 'submission.csv'
submission_fh = open(submission_f, 'wb')
submission.to_csv(submission_fh, sep=',', header='true')
submission_fh.close()